---
title: "Energy Insecurity Dashboard"
output: html
echo: false
---

In [ ]:
import hvplot.pandas
import pandas as pd
import panel as pn
import geopandas as gpd
import holoviews as hv
import numpy as np
from shapely import wkt

In [ ]:
disconnections = pd.read_csv("data\CA Zip Code ALK Data.csv")
zip_codes = pd.read_csv("data\San_Francisco_ZIP_Codes_20241221.csv")

In [ ]:
zip_codes['geometry'] = zip_codes['geometry'].apply(wkt.loads)

zip_codes_gdf = gpd.GeoDataFrame(zip_codes, geometry='geometry')

zip_codes_gdf.set_crs("EPSG:4326", inplace=True) 

disconnections['zip_code'] = disconnections['zip_code'].astype(str)
zip_codes_gdf['zip_code'] = zip_codes_gdf['zip_code'].astype(str)

merge = disconnections.merge(zip_codes_gdf, on="zip_code", how='inner')

disconnections_gdf = gpd.GeoDataFrame(merge, geometry="geometry")

month_mapping = {
    'Jan': 'Jan', 'January': 'Jan',
    'Feb': 'Feb', 'February': 'Feb',
    'Mar': 'Mar', 'March': 'Mar',
    'Apr': 'Apr', 'April': 'Apr',
    'May': 'May',
    'June': 'Jun', 'Jun': 'Jun',
    'July': 'Jul', 'Jul': 'Jul',
    'Aug': 'Aug', 'August': 'Aug',
    'Sep': 'Sep', 'Sept': 'Sep', 'September': 'Sep',
    'Oct': 'Oct', 'October': 'Oct',
    'Nov': 'Nov', 'November': 'Nov',
    'Dec': 'Dec', 'December': 'Dec'
}

# Normalizing the date format 
disconnections_gdf['month'] = disconnections_gdf['month'].map(month_mapping)

# Converting cleaned month names to numeric
disconnections_gdf['month'] = pd.to_datetime(disconnections_gdf['month'], format='%b').dt.month


disconnections_gdf['date'] = pd.to_datetime(disconnections_gdf[['year', 'month']].assign(day=1))
disconnections_gdf['formatted_date'] = disconnections_gdf['date'].dt.strftime('%Y-%b')

In [ ]:
pn.extension()
from bokeh.models import LinearColorMapper
from bokeh.palettes import Purples

In [ ]:
date_slider = pn.widgets.DateSlider(
    name='Select Date',
    start=disconnections_gdf['date'].min(),
    end=disconnections_gdf['date'].max(),
    value=disconnections_gdf['date'].max(),
    step=1,
    format='%Y-%b'
)

# Define a callback function that updates the map based on the selected date
def update_map(selected_date):
    # Filter the data based on the selected date
    filtered_data = disconnections_gdf[disconnections_gdf['date'] == selected_date]
    
    # Plot the data on a map
    map_plot = filtered_data.hvplot(
        geo=True, 
        tiles='CartoDark', 
        color='disconnection_rate', 
        cmap='Purples', 
        colorbar=True, 
        size=10,
        hover_cols=['ZIP_Code', 'disconnection_rate', 'disconnection_count'], 
        width=800, 
        height=600
    )
    
    return map_plot

# Set up the dashboard layout with the map and the slider
dashboard = pn.Column(
    date_slider,
    pn.bind(update_map, selected_date=date_slider)
)

# Display the dashboard
dashboard.save("dashboard.html", embed=True)

cat('<iframe src="dashboard.html" width="100%" height="800px"></iframe>')